# Learning with PyThaiNLP

**Text Preprocessing Pipeline**
1.Tokenization แบ่งคำ หน่วยย่อยๆ เรียกว่า Tokens ส่วนคำจะเรียกว่า Words , เครื่องหมายวรรคตอน หรือสัญลักษณ์อื่นๆ
|ภาษา|ข้อความเดิม|Token|
|---|---|---|
|english|I love NLP.| ```I , love , NLP , .```|
|thai|ฉันชอบเรียน NLP| ```ฉัน , ชอบ , เรียน , NLP```|

2.Normalization การปรับรูปแบบข้อความให้เป็น มาตรฐานเดียวกัน เพือให้คอม มองว่าเป็น คำเดียวกันแต่เขียนต่างกัน เช่น Case Folding เปลี่ยน Lowercase , Uppercase เช่น NLP , nlp มองเป็น nlp

3.Stop word removal กำจัดคำที่ไม่สำคัญ คือ การลบคำที่ไม่ค่อยมีความหมายต่อการวิเคราะห์ออกไป
English -> the , a , is , of ,and , in
Thai -> คือ , อยู่ , ที่ , นี้ , ฉัน , เรา , ของ

4.Stemming & Lemmatization หาคำมูลและรากศัพท์

Stemming คือตัดส่วนท้ายของคำออกตามกฏทั่วไป อาจจะไม่มีความหมายจริง
running -> runn

Lemmatization แปลงคำให้อยู่รูปศัพท์ตั้งต้นหลักไวยากรณ์ ได้ความหมายจริง
ran , running -> run
is , are , was -> be


NLP มี Tool ดังนี้
NLTK พื้นฐาน ได้ -> Tokenization , Stop Word Removal , Stemming/Lemmatization,POS Tagging

SpaCy เน้นประสิทธิภาพความเร็วสูง เหมาะกับงาน Production -> Tokenization , NER , Dependency Parsing
PyThaiNLP สำหรับภาษาไทย -> Tokenization คำไทย , Normaliation จัดการวรรณยุกต์ และสระเกิน


In [1]:
from pythainlp import word_tokenize

text = "ผมกำลังวิ่งในสวนรถไฟตอนเช้าแล้วก็เผลอทำโทรศัพท์มือถือราคา 2,000,000 บาท ตกหายไป"

tokens = word_tokenize(text,engine="newmm")
print("Token :", tokens)


Token : ['ผม', 'กำลัง', 'วิ่ง', 'ใน', 'สวน', 'รถไฟ', 'ตอนเช้า', 'แล้วก็', 'เผลอ', 'ทำ', 'โทรศัพท์มือถือ', 'ราคา', ' ', '2,000,000', ' ', 'บาท', ' ', 'ตกหาย', 'ไป']


In [2]:
# Normalization
tokens_filtered = [t for t in tokens if t.strip()]
print("Token (ลบช่องว่าง)", tokens_filtered)

Token (ลบช่องว่าง) ['ผม', 'กำลัง', 'วิ่ง', 'ใน', 'สวน', 'รถไฟ', 'ตอนเช้า', 'แล้วก็', 'เผลอ', 'ทำ', 'โทรศัพท์มือถือ', 'ราคา', '2,000,000', 'บาท', 'ตกหาย', 'ไป']


In [4]:
# Handle Number
import re
def normalize_numbers(tokens):
    normalized = []
    for token in tokens:
        if re.match(r'^-?(\d{1,3}(,\d{3})*|\d+)(\.\d+)?$',token):
            normalized.append('<NUM>')
        else:
            normalized.append(token)
    return normalized

tokens_normalized = normalize_numbers(tokens_filtered)
print("Token Number :", tokens_normalized)

Token Number : ['ผม', 'กำลัง', 'วิ่ง', 'ใน', 'สวน', 'รถไฟ', 'ตอนเช้า', 'แล้วก็', 'เผลอ', 'ทำ', 'โทรศัพท์มือถือ', 'ราคา', '<NUM>', 'บาท', 'ตกหาย', 'ไป']


In [7]:
# Stop Word Removal
from pythainlp.corpus import thai_stopwords
stop_words = thai_stopwords()

def remove_stopwords(tokens):
    return [t for t in tokens if t not in stop_words]
tokens_no_stop = remove_stopwords(tokens_normalized)
print("Token Stop word :")
print(tokens_no_stop)

Token Stop word :
['ผม', 'วิ่ง', 'สวน', 'รถไฟ', 'ตอนเช้า', 'แล้วก็', 'เผลอ', 'ทำ', 'โทรศัพท์มือถือ', 'ราคา', '<NUM>', 'บาท', 'ตกหาย']


In [13]:
# Lemmatization แปลงกลับไปรากศัพท์ โดยมองว่าเป็นความหมายเดียวกัน
from pythainlp.tag import pos_tag
pos_tags = pos_tag(tokens_no_stop,engine='perceptron')
print("Token POS Tag")
print(pos_tags)


Token POS Tag
[('ผม', 'PPRS'), ('วิ่ง', 'VACT'), ('สวน', 'NCMN'), ('รถไฟ', 'NCMN'), ('ตอนเช้า', 'ADVN'), ('แล้วก็', 'JCRG'), ('เผลอ', 'NCMN'), ('ทำ', 'VACT'), ('โทรศัพท์มือถือ', 'NCMN'), ('ราคา', 'NCMN'), ('<NUM>', 'PUNC'), ('บาท', 'NCMN'), ('ตกหาย', 'NCMN')]


In [ ]:
# Frequency-Based Methods
# Bag-of-Word (Bow)
# เอกสารเป็นถุก โดยไม่สนใจลำดับของคำ แต่สนใจว่าคำนั้น ปรากฏมากี่ครั้ง
import pandas as pd
bags_word = {}
for i in pos_tags:
    if i[0] not in bags_word:
        bags_word[i[0]] = 1
    else:
        bags_word[i[0]] +=1
df_word = pd.DataFrame(list(bags_word.items()),columns=['Word','Frequency'])
display(df_word) 
# TF-IDF (Term Frequency-Inverse Document Frequency)
# ตล้าย BOW แต่จะทำการ ถ่วงน้ำหนัก ความถี่ของคำเพื่อเน้นคำที่สำคัยต่อเอกสาารนั้นๆ 
# TF-IDF มีส่วนแรก term frequency (TF) = TF(t,d) = จำนวนคร้ังที่ ค่า t ปรากฏในเอกสาร d
# Inverse Document Frequency (IDF) คือค่าน้ำหนักผกผัน ให้กับคำ ที่ปรากฏหลายๆเอกสาร คำทั่วไป
# จะมีค่า IDF ต้ำและค่าที่เอกสารเดี่ยวๆ ค่าเฉลี่ยจะมี สูง
# IDF(t,D) = log(จำนวนเอกสารทั้งหมด/จำนวนเอกสารที่มี่ ค่า t ปรากฏ)
# TF-IDF คือ การนำ TF มาคูณ กับ IDF

,Word,Frequency
0,ผม,1
1,วิ่ง,1
2,สวน,1
3,รถไฟ,1
4,ตอนเช้า,1
5,แล้วก็,1
6,เผลอ,1
7,ทำ,1
8,โทรศัพท์มือถือ,1
9,ราคา,1


In [ ]:
#แปลงความหมาย Semantic-Based Method
# เน้น word embedding
# Word2Vec & GloVe
# คำที่ปรากฏความหมายใกลเคี้ยงกัน
# Word2Vec 1,Continuous Bag-of-Words ใช้คำโดยรอบข้างเพื่่อทำนายคำตรงกลาง 2,Skip-gram ใช้คำตรงกลางเพื่อทำนายรอบข้าง
# GloVe ใช้สถิติร่วมกับการเรียนรู้เชิงลึก โดยสร้าง vector จาก อัตราส่วนของการปรากฏร่วมกัน Co-occurrnece Matrix
# | คำ | vector 3d | meaning |
# | โทรศัพท์มือถือ | [0.8,-0.3,0.1]| (อุปกรณ์, อิเล็กทรอนิกส์ , พกพา)
# | แท็ยแล็ต | [0.7,-0.4,0.2]| (อุปกรณ์, อิเล็กทรอนิกส์ , พกพา)
# | วิ่ง | [-0.5,0.9,-0.1]| (กิจกรรม, กริยา , เคลือนไหว)
# Contextualized Embeddings (BERT/LLMs)
# ใช้ architecture Transformer , attention Mechanism เพื่อสร้าง vector
# BERT from transformer ไปข้อความทั้งหมดไปหน้า และย้อนกลับเพื่อสร้าง เวกเตอร์
# LLM -> GPT | T5 อาศัย Transformer และ Contextual Embeddings เพื่อประมวลภาษา


# Text Classification

In [34]:
# Data Preparation
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
corpus = [
    'สินค้าดีมาก การจัดส่งรวดเร็วทันใจ บริการดีเยี่ยม',
    'รสชาติกาแฟอร่อย แต่ราคาแพงไปนิดหน่อย',
    'การบริการแย่มาก พนักงานหน้าบึ้ง ไม่ประทับใจเลย',
    'พัสดุถูกส่งมาตามกำหนดเวลา ไม่มีปัญหาอะไร',
    'สถานที่สวยงาม บรรยากาศดี เหมาะกับการพักผ่อน',
    'ฉันคิดว่าแอปพลิเคชันนี้ใช้งานง่ายและเป็นประโยชน์มาก',
    'คุณภาพสินค้าต่ำกว่าที่คาดไว้ รู้สึกผิดหวัง',
    'หนังเรื่องนี้ธรรมดา ไม่ดีไม่แย่ ดูได้เรื่อยๆ',
    'รถยนต์คันใหม่ประหยัดน้ำมันได้ดีมาก ถูกใจจริงๆ',
    'มีปัญหาในการใช้งานระบบ แต่ได้รับการแก้ไขรวดเร็ว',
]
Y = [1,0,2,0,1,1,2,0,1,0]
df_text = pd.DataFrame(list(corpus),columns=["Text"])
df_text['Label'] = Y
# TF-IDF Vector
def custom_tokenizer(text):
    return word_tokenize(text,engine='newmm')
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)
X = vectorizer.fit_transform(df_text['Text'])

c:\Users\home\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,df_text['Label'],test_size=0.3,random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(X_train , y_train)
y_pred = model.predict(X_test)
display(y_pred) # โมเดล เดาว่า ลบ ทัศนคติเชิงลบ
# 0 กลาง 1 บวก 2 ลบ

array([0, 0, 0], dtype=int64)

In [36]:
from sklearn.metrics import classification_report
target_names = ['กลาง (0)','บวก (1)','ลบ (2)']
expected_labels = [0,1,2]
print(classification_report(y_test,y_pred,labels=expected_labels,target_names=target_names))

              precision    recall  f1-score   support

    กลาง (0)       0.33      1.00      0.50         1
     บวก (1)       0.00      0.00      0.00         2
      ลบ (2)       0.00      0.00      0.00         0

    accuracy                           0.33         3
   macro avg       0.11      0.33      0.17         3
weighted avg       0.11      0.33      0.17         3



c:\Users\home\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\home\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\home\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

# Named Entity Recognition (NER)

Named Entity Recognition (NER) การระบุเอนทิตี้ที่มีชื่อ
NER คือการค้นหาและจำแนก เอนทิที้ที่มีชื่อในข้อความ เช่น ชื่อคน สถานที่ องค์กร เวลา จำนวนเงิน
โดย ช่วยทำให้เข้าใจว่า who what where when  ในประโยคทำให้สำคัญต่อการทำ information Extraction (การสกัดข้อมูล) และ Question Answering 
|Entity|Example|Label|
|---|---|---|
|PERSON|สมชาย ลิซ่า|PER|
|ORG|ธกรุงเทพ , google|ORG|
|LOCATION|กรุงเทพ สวนลุม|LOC|
|DATE/TIME|วันที่ 25 พฤษภาคม เมือวานนี้|DATE/TIME|
|MONEY|20,000 บาท , 5 ล้านดอลลาร์|MONEY|

NER model ในปัจจุบัน แทนที่จะใช้ความถี่ BoW/TF-IDF เหมือน Text classification พื้นฐาน งาน NER ต้องใช้โมเดลที่สามารถตวสจจับบริบทได้ เช่น RNN / Bi-LSTM-CRF เป็น
โมเดลในอดีต เนื่องจากสามารถอ่านบริบทของคำในลำดับ (Sequence) ได้จากทั้งสองทิศทาง (ไปหน้า - หลัง)
Transformer Model (BERT/XLM-R) เป็นวิธีที่ทันสมัยที่สุดในปัจจุบัน โดยใช้ transformer เพื่อสร้าง contextual embeddings ที่แม่นยำสูงและนำไป fine-tune สำหรับงาน NER

In [12]:
from pythainlp.tag import NER
ner = NER()
text_thai = "คุณสมบัติของท่านสามารถใช้เงินที่ฝากไว้กับธนาคารกรุงเทพในการลงทุนในตลาดหลักทรัพย์ได้"
entities = ner.tag(text_thai)
print("NER :")
print([f"{i}" for i in entities])

NER :
["('คุณสมบัติ', 'O')", "('ของ', 'O')", "('ท่าน', 'O')", "('สามารถ', 'O')", "('ใช้', 'O')", "('เงิน', 'O')", "('ที่', 'O')", "('ฝาก', 'O')", "('ไว้', 'O')", "('กับ', 'O')", "('ธนาคาร', 'B-ORGANIZATION')", "('กรุงเทพ', 'I-ORGANIZATION')", "('ใน', 'O')", "('การลงทุน', 'O')", "('ใน', 'O')", "('ตลาด', 'O')", "('หลักทรัพย์', 'O')", "('ได้', 'O')"]


# Text Summarization สรุปแบบดึงมา (Extractive Summarization)

In [14]:
# คือการดึงประโยคสำคัญ จากเอกสารต้นฉบับมาเรียงต่อเป็น ข้อความสรุป
# วิิธีไม่ได้สร้างประโยคใหม่ แต่เน้นค้นหาประโยคที่มีความสำคัญสูงสุด โดยใช้ Sentence Scoring
# 1.TF 2. TF-IDF 3.Sentence Position
# ซึ่งมีเทคนิค TextRank
# โดยจะสราา้ง Graph Construction มี Nodes แต่ละประโยค และ Edges เส้นเชื่อมระหว่างสอง nodes
# Score calcculation node ที่เชื่อมกันจำนวนมาก หรือ node สำคัญ จะได้รับคะแนนสูงกว่า 
# จะใช้ algorithm วนซ้ำจนกว่าคะแนนของทุก node จะคงที่
# สร้างสรุป โดยเลือก node ที่ได้คะแนนสูงสุดตามจำนวนที่ต้องการสรุป เช่น 3 ประโยคแรก
from transformers import pipeline

# สมมติว่ามีโมเดล Transformer ภาษาไทยที่ถูก Fine-tune มาสำหรับ Summarization
# (เช่น 'seonglae/mBart-large-th-summarization' หรือ LLM ตัวอื่น)
try:
    summarizer = pipeline(
        "summarization", 
        model="Nopphakorn/mt5-small-thaisum-512-title", 
        tokenizer="Nopphakorn/mt5-small-thaisum-512-title"
    )
    
    long_text = "การประชุมสุดยอดผู้นำอาเซียนครั้งที่ 43 ได้เริ่มต้นขึ้นในกรุงจาการ์ตา อินโดนีเซีย เมื่อวันที่ 5 กันยายน 2566 โดยมีวาระสำคัญคือการเสริมสร้างความร่วมมือทางเศรษฐกิจและการแก้ไขปัญหาวิกฤตการณ์ในเมียนมา นายกรัฐมนตรีของไทยได้เดินทางเข้าร่วมเพื่อเน้นย้ำถึงความสำคัญของการค้าเสรีและการลงทุนในภูมิภาค"
    
    # สรุปแบบ Abstractive (เพราะใช้ Transformer)
    summary_result = summarizer(long_text, max_length=50, min_length=10, do_sample=False)
    
    print("\nผลลัพธ์ Abstractive Summarization:")
    print(summary_result[0]['summary_text'])
    
except ImportError:
    print("\n---")
    print("ไม่สามารถรันตัวอย่าง Transformer ได้ กรุณาติดตั้งไลบรารี transformers และ PyTorch/TensorFlow")
    print("---")

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

c:\Users\home\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\home\.cache\huggingface\hub\models--Nopphakorn--mt5-small-thaisum-512-title. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



ผลลัพธ์ Abstractive Summarization:
นายกรัฐมนตรีของไทยเดินทางเข้าร่วมประชุมสุดยอดผู้นําอาเซียนครั้งที่ 43 ด้วยการเน้นย้ําถึงความสําคัญของการค้าเสรีและการลงทุนในภูมิภาคอาเซียน


# Basic Word Thai with PyThaiNLP

In [1]:
import pythainlp
pythainlp.__version__

'5.1.2'

In [ ]:
print("ตัวอักษรไทย")
print(pythainlp.thai_characters)
print("จำนวนตัวอักษร : ",len(pythainlp.thai_characters))
print("พยัญชนะไทย")
print(pythainlp.thai_consonants)
print("จำนวนตัวอักษร : ",len(pythainlp.thai_consonants))
print("สระ")
print(pythainlp.thai_vowels)
print("จำนวนตัวอักษร : ",len(pythainlp.thai_vowels))
print("วรรณยุกต์")
print(pythainlp.thai_tonemarks)
print("จำนวนตัวอักษร : ",len(pythainlp.thai_tonemarks))
print("เครื่องหมายที่เกี่ยวข้อง")
print(pythainlp.thai_signs)
print("จำนวนตัวอักษร : ",len(pythainlp.thai_signs))
print("เครื่องหมายวรรคตอน")
print(pythainlp.thai_punctuations)
print("จำนวนตัวอักษร : ",len(pythainlp.thai_punctuations))
print("เลขไทย")
print(pythainlp.thai_digits)
print("จำนวนตัวอักษร : ",len(pythainlp.thai_digits))
print("สัญลักษณ์ สกุลเงินไทย ->", pythainlp.thai_symbols)
print("รวมอักษรไทย")
print(pythainlp.thai_letters)
print("จำนวนตัวอักษร : ",len(pythainlp.thai_letters))
print("เช็คว่า , อยู่ใน set character ไหม : ",(',' in pythainlp.thai_characters))
print("เช็คว่า ๑ อยู่ใน set character ไหม : ",('๑' in pythainlp.thai_characters))


In [ ]:
import pythainlp.util

print(pythainlp.util.isthai("ก"))
print(pythainlp.util.isthai("สวัสดีตอนเช้า."))
print(pythainlp.util.isthai("สวัสดีตอนเช้า กินข้าวหรือยัง?"))
print(pythainlp.util.isthai("สวัสดีตอนเช้า กินข้าวหรือยัง?",ignore_chars="?"))
print(pythainlp.util.isthai("สวัสดีตอนเช้า กินข้าวหรือยัง?",ignore_chars=" ?"))
print(pythainlp.util.isthai("ดัชนีช่วงเช้าปิดที่ระดับ 60,596.53 จุด เพิ่มขึ้น 18.01 จุด หรือ 2.51% มูลค่าการซื้อขาย 8.95 หมื่นล้านบาท",ignore_chars=" 01234567890,.%"))

In [ ]:
print(pythainlp.util.countthai("แมวสีสวาท"))
print(pythainlp.util.countthai("แมว cat"))
print(pythainlp.util.countthai("วันอาทิตย์ที่ 24 มีนาคม 2562"))
print(pythainlp.util.countthai("วันอาทิตย์ที่ 24 มีนาคม 2562",ignore_chars=""))

In [ ]:
from pythainlp.util import collate
thai_words = ["หมู","ไก่","ช้าง","ม้า","ควาย"]
collate(thai_words)

In [ ]:
collate(thai_words,reverse=True)

In [ ]:
# ไทย ผสมตัวเลข
thai_words = ["หมูแดง", "หมูทอด", "หมู 3 ชั้น", "หมูกรอบ", "หมูปิ้ง", "หมูตุ๋น", "หมูแฮม", "Suckling pig", "Pork chop"]
collate(thai_words)

In [ ]:
import datetime
from pythainlp.util import thai_strftime

date = datetime.datetime(2020,12,31,23,59)
fmt = "%Aที่ %-d %B พ.ศ. %Y เวลา %H:%M น."
thai_strftime(date,fmt)

In [ ]:
fmt = "%d-%b-%y"
thai_strftime(date,fmt)

In [ ]:
fmt = "%c"
thai_strftime(date,fmt)

In [ ]:
fmt = "%v"
thai_strftime(date,fmt)

In [ ]:
fmt = "%Aที่ %-d %B พ.ศ. %Y เวลา %H:%M น. (%a %d-%b-%y)"
thai_strftime(date,fmt,thaidigit=True)

In [ ]:
from pythainlp.util import time_to_thaiword
print(time_to_thaiword("00:01:19"))
print(time_to_thaiword("00:00:01"))
print(time_to_thaiword("00:10:00"))
print(time_to_thaiword("01:00:00"))
print(time_to_thaiword("00:00:00"))
print(time_to_thaiword("23:59:00"))

In [ ]:
print(time_to_thaiword("19:30:01",fmt="24h"))
print(time_to_thaiword("19:30:01",fmt="6h"))
print(time_to_thaiword("05:00:07",fmt="24h"))
print(time_to_thaiword("05:00:07",fmt="6h"))

In [ ]:
print(time_to_thaiword("08:28:59",fmt="6h"))
print(time_to_thaiword("08:28:59",fmt="m6h"))

In [ ]:
print(time_to_thaiword("11:48:59",precision="m"))

In [ ]:
print(time_to_thaiword("11:00:00",fmt="24h"))
print(time_to_thaiword("11:00:00",fmt="24h",precision="s"))
print(time_to_thaiword("11:00:00",fmt="24h",precision="m"))
print(time_to_thaiword("11:00:00",fmt="24h",precision=None))

In [ ]:
print(time_to_thaiword("11:48:00",fmt="6h"))
print(time_to_thaiword("11:48:00",fmt="6h",precision="s"))
print(time_to_thaiword("11:48:00",fmt="6h",precision="m"))
print(time_to_thaiword("11:48:00",fmt="6h",precision=None))

In [ ]:
print(time_to_thaiword("07:00:00",fmt="m6h"))
print(time_to_thaiword("07:00:00",fmt="m6h",precision="s"))
print(time_to_thaiword("07:00:00",fmt="m6h",precision="m"))
print(time_to_thaiword("07:00:00",fmt="m6h",precision=None))

In [ ]:
import datetime
time = datetime.time(23,59,58)
time_to_thaiword(time)

In [ ]:
time = datetime.datetime(1980,12,31,23,58,57)
time_to_thaiword(time,fmt="6h",precision="m")

In [1]:
from pythainlp.util import num_to_thaiword , thaiword_to_num , bahttext
print(num_to_thaiword(123456789))
print(num_to_thaiword(0))
print(thaiword_to_num("สองพันล้านล้านสี่หมื่นหกพันเจ็ดร้อยยี่สิบสอง"))
print(bahttext(12345.67))
print(bahttext(10000000000.00))
print(bahttext(0))
print(bahttext(0.01))
print(bahttext(0.001))

หนึ่งร้อยยี่สิบสามล้านสี่แสนห้าหมื่นหกพันเจ็ดร้อยแปดสิบเก้า
ศูนย์
2000000000046722
หนึ่งหมื่นสองพันสามร้อยสี่สิบห้าบาทหกสิบเจ็ดสตางค์
หนึ่งหมื่นล้านบาทถ้วน
ศูนย์บาทถ้วน
ศูนย์บาทหนึ่งสตางค์
ศูนย์บาทถ้วน


In [ ]:
from pythainlp.soundex import lk82 , metasound , udom83
print(lk82("ค่า")== lk82("ข้า"))
print(lk82("การ")== lk82("กาล"))
print(lk82("เพชรรัช")== lk82("เพชรรัศม์"))
print(metasound("สวิทซ์")== metasound("สวัส"))
print(udom83("ค่า")== udom83("ข้า"))

In [ ]:
texts = ["ค่า", "ข้า", "ฆ่า", "การ", "กาล", "การณ์", "เพ็ชรรัตน์ ", "เพชรรัตติ์", "รัก", "รักษ์"]

for text in texts:
    print(f"Text: {text}, lk82:, {lk82(text)}, udom83: {udom83(text)}, metasound: {metasound(text)}")

# SpaCy_NLP

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...


True

Stemming คึอ กระบวนการตัดส่วนท้ายคำ แบบหยาบๆ Heuristic ซึ่งได้ผล พอตวร สำเหรับ อังกฤษ และทำให้คำลดลง ซึ่งเป็นการตัดแบบอาจจะเสียความหมายไปเลย
Lemmatization คือแปลง work ด้วยรายการศัพท์ใน dict ส่วนใหญ่ จะตัดส่วนท้ายของคำให้ คงรูปพื้นฐาน เรียกว่า Lemma

In [4]:
# สร้าง WordNetLemmatizer , PorterStemmer เป็น Stemmer , Lemmatizer อังกฤษ
from nltk import stem
wnl = stem.WordNetLemmatizer()
porter = stem.PorterStemmer()

In [5]:
def print_lemma_stem(word_list):
    for word in word_list:
        print(f'{word:12} ==> {wnl.lemmatize(word):12}\t{porter.stem(word):12}')

In [6]:
word_list = ['foot', 'feet', 'foots', 'footing']
print_lemma_stem(word_list)

foot         ==> foot        	foot        
feet         ==> foot        	feet        
foots        ==> foot        	foot        
footing      ==> footing     	foot        


In [7]:
word_list = ['fly', 'flies', 'flying', 'flew', 'flown']
print_lemma_stem(word_list)

fly          ==> fly         	fli         
flies        ==> fly         	fli         
flying       ==> flying      	fli         
flew         ==> flew        	flew        
flown        ==> flown       	flown       


In [8]:
word_list = ['organize', 'organized', 'organizing']
print_lemma_stem(word_list)

organize     ==> organize    	organ       
organized    ==> organized   	organ       
organizing   ==> organizing  	organ       


In [9]:
word_list = ['universe', 'university', 'universal']
print_lemma_stem(word_list)

universe     ==> universe    	univers     
university   ==> university  	univers     
universal    ==> universal   	univers     


In [10]:
word_list = "The formatting operations described here exhibit a variety of quirks that lead to a number of common errors.".split()
print_lemma_stem(word_list)

The          ==> The         	the         
formatting   ==> formatting  	format      
operations   ==> operation   	oper        
described    ==> described   	describ     
here         ==> here        	here        
exhibit      ==> exhibit     	exhibit     
a            ==> a           	a           
variety      ==> variety     	varieti     
of           ==> of          	of          
quirks       ==> quirk       	quirk       
that         ==> that        	that        
lead         ==> lead        	lead        
to           ==> to          	to          
a            ==> a           	a           
number       ==> number      	number      
of           ==> of          	of          
common       ==> common      	common      
errors.      ==> errors.     	errors.     


In [12]:
# Lemmatization using Spacy
import spacy
nlp = spacy.load("en_core_web_sm")

In [13]:
def print_lemma(sentence):
    sentence = nlp(sentence)
    for word in sentence:
        print(f'{word.text:12} \t==> {word.lemma_:12}')

In [15]:
print_lemma('car cars')

car          	==> car         
cars         	==> car         


In [16]:
print_lemma('organize organized organizes organizing organization organizations')

organize     	==> organize    
organized    	==> organized   
organizes    	==> organize    
organizing   	==> organize    
organization 	==> organization
organizations 	==> organization
